# Trabalho Final

**Aluno**:

**Matrícula**:


## 03 - Tratamento da base *formacao.csv*

Descreve a base aqui com suas palavras.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

formacao = pd.read_csv("bases/formacao.csv")

# Análise inicial da qualidade dessa base
print("Quantidade de registros: ", len(formacao))
print("Quantidade de valores distintos de perfil_id: ", formacao.perfil_id.nunique())
print("Dados faltantes em degree: ", len(formacao.loc[formacao.degree.isnull()]))
print("Dados faltantes em date: ", len(formacao.loc[formacao.date.isnull()]))
print("Dados faltantes em major: ", len(formacao.loc[formacao.major.isnull()]))
print("Dados faltantes em school: ", len(formacao.loc[formacao.school.isnull()]))
print("Dados faltantes em degree + major: ", len(formacao.loc[(formacao.degree.isnull()) & (formacao.major.isnull())]))
print("Dados faltantes em degree + major + school: ", len(formacao.loc[(formacao.degree.isnull()) & (formacao.major.isnull()) & (formacao.school.isnull())]))
print("Valores distintos em degree: ", len(formacao.groupby('degree').nunique()))
print("Valores distintos em major: ", len(formacao.groupby('major').nunique()))
print("Valores distintos em school: ", len(formacao.groupby('school').nunique()))

In [ ]:
formacao.head(5)

## 03.1 - Tratamento das data

Um primeiro tratamento feito será o de data. A coluna **date** que indica o íncio e o fim do curso e será quebrada em duas colunas numéricas (**inicio** e **fim**), e ainda numa terceira coluna indicando a duração estimada do curso em anos baseado na informação obtida (**duracao**).

## 03.2 - Tratamento da coluna de grau de escolaridade

Uma vez que a coluna **degree** não é muito precisa, uma coluna **grau** será criada. Ela começará vazia e será armazenado a inferência do grau de formação baseado na análise das demais colunas.

Uma primeira estratégia será avaliar o nome do **degree** para inferir se ele contém algum fragmento que possa indicar algum dos seguintes graus de escolaridade, representados por números: **Ensino Médio**=1, **Técnico**=2, **Superior**=3 ou **Pós-Graduação**=4.

Para isso, utilize a função [`contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html).

Para os demais casos do grau de escolaridade não cobertos pela análise do texto, deverá ser feita uma análise mais profunda. Tente encontrar outros padrões para definir o grau de escolaridade, por exemplo, o nome da instituição de ensino (campo **school**) ou, então, o tempo de duração.

## 03.3 - Tratamento da coluna de grau de escolaridade

A coluna **school** apresenta uma vasta de gama de categorias. Porém numa análise visual dos seus diferentes valores é possível ver que algumas instituições de ensino apresentam variações de nome. Foi feita uma substituição para agrupar esses casos num único nome. O resultao do tratamento é armazenado na nova coluna **instituicao**.

## 03.4 - Área de Formação

Agrupe as formações em grupos mais macros, por exemplo: Engenharia de Alimento, Engharia Elétrica estariam no grupo de engenharias, já Ciência da Computação, Sistemas de Informação poderiam estar no grupo de T.I. Crie áreas que façam sentido, agrupe os dados criando uma nova coluna no dataframe chamada area.

## 03.5 - Escolaridade

Uma informação consolidada de cada funcionário que pode ser derivada dessa base é a sua escolaridade (**escolaridade**), que é pode ser obtida pegando o maio grau daquele perfil. Gere uma base à parte contendo essa informação.

## 03.6 Salvando Base Tratada

A base tratada será gravada no arquivo **formacao-tratado.csv**.

A base consolidadada com uma informação por registro será gravada no arquivo **perfil-formacao.csv** contendo as colunas: **perfil_id**, **escolaridade**, **escolaridade_numero**, **area**, **duracao**.

In [ ]:
# Grava a formação tratada em arquivo
formacao_tratado.to_csv('bases/formacao-tratado.csv')

# Grava o consolidade da formação por perfil em arquivo
formacao_perfil = pd.concat([escolaridade, area], axis=1)
formacao_perfil.to_csv('bases/perfil-formacao.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)
* [Continuar para o próximo documento](04-tratamento-base-habilidades.ipynb)